In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

import gym
from tqdm import tqdm_notebook
import numpy as np
from collections import deque

In [ ]:
#discount factor for future utilities
DISCOUNT_FACTOR = 0.99
gamma = 0.99
#number of episodes to run
NUM_EPISODES = 5000

#max steps per episode
MAX_STEPS = 10000



#device to run model on
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
class PolicyNetwork(nn.Module):
  def __init__(self, observation_space, action_space,seed,layer_size):
        super(PolicyNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.input_layer = nn.Linear(observation_space, layer_size)
        self.output_layer = nn.Linear(layer_size, action_space)
  def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(x)
        actions = self.output_layer(x)
        action_probs = F.softmax(actions, dim=-1)
        return action_probs

In [ ]:

class StateValueNetwork(nn.Module):
    def __init__(self, observation_space,seed,layer_size):
        super(StateValueNetwork, self).__init__()
        self.input_layer = nn.Linear(observation_space, 128)
        self.output_layer = nn.Linear(128, 1)
    def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(x)
        state_value = self.output_layer(x)
        return state_value

In [ ]:
def process_rewards(rewards,gamma):
    G = []
    total_r = 0
    for r in reversed(rewards):
        total_r = r + total_r * DISCOUNT_FACTOR
        G.insert(0, total_r)
    G = torch.tensor(G)
    #G = (G - G.mean())/G.std()
    return G

In [ ]:
def train_value(G, state_vals, optimizer):
    val_loss = F.mse_loss(state_vals, G)
    optimizer.zero_grad()
    val_loss.backward()
    optimizer.step()

In [ ]:
seed_list = [1,42,30,25,17]

In [ ]:
def reset_weights(model):
    for layer in model.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()

In [ ]:
env = gym.make('Acrobot-v1')


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
def avg_over_5_runs(params):
  avg_regret = 0
  for i in range(5):
    regret = 0
    seed = seed_list[i]
    #Init network
    policy_network = PolicyNetwork(env.observation_space.shape[0], env.action_space.n,seed,params["network_size"])
    stateval_network = StateValueNetwork(env.observation_space.shape[0],seed,params["network_size"])
    reset_weights(policy_network)
    reset_weights(stateval_network)

    #Init optimizer
    policy_optimizer = optim.Adam(policy_network.parameters(), lr=params["LR"])
    stateval_optimizer = optim.Adam(stateval_network.parameters(), lr=params["LR"])

    ep = 0
    action_space = np.arange(env.action_space.n)
    total_rewards1 = []
    while ep < NUM_EPISODES:
      state = env.reset()
      states = []
      rewards = []
      actions = []
      done = False
      while done == False:
        state = torch.from_numpy(state).float().unsqueeze(0)[0]
        action_probability = policy_network.forward(state).detach().numpy()
        if np.isnan(action_probability).any():
          print("oh",avg_regret/i)
          return avg_regret / i
        '''
        if np.isnan(action_probability).any():
          print("hi")
          state = env.reset()
          states = []
          rewards = []
          actions = []
          done = False
          state = torch.from_numpy(state).float().unsqueeze(0)[0]
          action_probability = policy_network.forward(state).detach().numpy()


        action_probability[np.isnan(action_probability)] = 0.0  # Replace NaN with 0.0'''

        action = np.random.choice(action_space,p=action_probability)
        state.detach()
        next_state,r,done,_ = env.step(action)
        states.append(state)
        rewards.append(r)
        actions.append(action)
        if done :
          break
        state = next_state
      total_rewards1.append(sum(rewards))
      G = process_rewards(rewards,gamma)
      G = torch.FloatTensor(G)

      rewards = torch.FloatTensor(rewards)

      policy_optimizer.zero_grad()
      state_vals = []

      for st in states:
          st = st.float().unsqueeze(0)[0]

          state_vals.append(stateval_network(st))
      state_vals = torch.stack(state_vals).squeeze()
      #print(state_vals)
      train_value(G, state_vals, stateval_optimizer)

      deltas = [gt - val for gt, val in zip(G, state_vals)]
      deltas = torch.tensor(deltas)
      logprob = [torch.log(policy_network.forward(states[i])) for i in range(len(deltas))]
      policy_loss = []
      for i in range(len(deltas)):

            d = deltas[i]

            lp = logprob[i][actions[i]]

            policy_loss.append(-d * lp)
      policy_optimizer.zero_grad()
      #print(policy_loss,len(actions))
      sum(policy_loss).backward()
      torch.nn.utils.clip_grad_norm_(policy_network.parameters(), 5)
      policy_optimizer.step()
      avg_rewards = np.mean(total_rewards1[-100:])
      regret = regret - avg_rewards

      ep +=1
      if ep%400 == 0:
          print("Ep:",ep,"last 100 episodes reward is  :",avg_rewards, end="\n")
    avg_regret = avg_regret + regret
    print("problem solved at episode",ep)

  return avg_regret / 5



In [ ]:
'''
bounds = [(1e-4, 1e-2)]

# Run the Bayesian optimization
res = gp_minimize(avg_over_5_runs, bounds, n_calls=10, random_state=0)
print(res)
# Print the best parameters
print(f"Best parameters: learning rate = {res.x[0]}")
'''

'\nbounds = [(1e-4, 1e-2)]\n\n# Run the Bayesian optimization\nres = gp_minimize(avg_over_5_runs, bounds, n_calls=10, random_state=0)\nprint(res)\n# Print the best parameters\nprint(f"Best parameters: learning rate = {res.x[0]}")\n'

In [ ]:

import matplotlib.pyplot as plt


In [ ]:
!pip install wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00


In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import wandb

def main():
    wandb.init(project="RLA2acrobot-withbase")
    #agent = TutorialAgent(state_size=state_shape,action_size = action_shape,seed = 0, config=wandb.config)
    score = avg_over_5_runs(wandb.config)
    wandb.log({"score": score})

# 2: Define the search space
sweep_configuration = {
    "method": "bayes",

    "metric": {"goal": "minimize", "name": "score"},
    "parameters": {
        "LR": {"max": 1e-3, "min": 1e-6},
        "network_size": {"values": [64,256,128]},
    },
}

# 3: Start the sweep
sweep_id = wandb.sweep(sweep=sweep_configuration, project="RLA2acrobot-withbase")

wandb.agent(sweep_id, function=main, count=10)

/usr/local/lib/python3.10/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)


Create sweep with ID: 46vsm7aa
Sweep URL: https://wandb.ai/deep_learning_duri/RLA2acrobot-withbase/sweeps/46vsm7aa


wandb: Agent Starting Run: enzmehgm with config:
wandb: 	LR: 0.0002534149211734855
wandb: 	network_size: 64
wandb: Currently logged in as: dhurilkun (deep_learning_duri). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Ep: 400 last 100 episodes reward is  : -477.89
Ep: 800 last 100 episodes reward is  : -286.55
Ep: 1200 last 100 episodes reward is  : -147.29
Ep: 1600 last 100 episodes reward is  : -122.49
Ep: 2000 last 100 episodes reward is  : -109.08
Ep: 2400 last 100 episodes reward is  : -103.09
Ep: 2800 last 100 episodes reward is  : -103.87
Ep: 3200 last 100 episodes reward is  : -95.36
Ep: 3600 last 100 episodes reward is  : -96.63
Ep: 4000 last 100 episodes reward is  : -92.11
Ep: 4400 last 100 episodes reward is  : -89.58
Ep: 4800 last 100 episodes reward is  : -87.24
problem solved at episode 5000
Ep: 400 last 100 episodes reward is  : -454.1
Ep: 800 last 100 episodes reward is  : -243.73
Ep: 1200 last 100 episodes reward is  : -153.51
Ep: 1600 last 100 episodes reward is  : -115.89
Ep: 2000 last 100 episodes reward is  : -109.84
Ep: 2400 last 100 episodes reward is  : -103.15
Ep: 2800 last 100 episodes reward is  : -95.16
Ep: 3200 last 100 episodes reward is  : -95.55
Ep: 3600 last 100 epi

...

Ep: 400 last 100 episodes reward is  : -227.11
Ep: 800 last 100 episodes reward is  : -112.36
Ep: 1200 last 100 episodes reward is  : -99.85
Ep: 1600 last 100 episodes reward is  : -84.31
Ep: 2000 last 100 episodes reward is  : -81.27
Ep: 2400 last 100 episodes reward is  : -85.33
Ep: 2800 last 100 episodes reward is  : -87.99
Ep: 3200 last 100 episodes reward is  : -87.31
Ep: 3600 last 100 episodes reward is  : -84.08
Ep: 4000 last 100 episodes reward is  : -83.91
Ep: 4400 last 100 episodes reward is  : -84.96
Ep: 4800 last 100 episodes reward is  : -82.89
problem solved at episode 5000
Ep: 400 last 100 episodes reward is  : -450.52
Ep: 800 last 100 episodes reward is  : -138.91
Ep: 1200 last 100 episodes reward is  : -94.64
Ep: 1600 last 100 episodes reward is  : -89.28
Ep: 2000 last 100 episodes reward is  : -86.27
Ep: 2400 last 100 episodes reward is  : -81.78
Ep: 2800 last 100 episodes reward is  : -84.5
Ep: 3200 last 100 episodes reward is  : -84.91
oh 3434.6409328847712


score,▁
score,3434.64093


wandb: Agent Starting Run: g2hestqw with config:
wandb: 	LR: 0.0004245149843280246
wandb: 	network_size: 128
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Ep: 400 last 100 episodes reward is  : -301.67
Ep: 800 last 100 episodes reward is  : -127.32
Ep: 1200 last 100 episodes reward is  : -99.07
Ep: 1600 last 100 episodes reward is  : -94.65
Ep: 2000 last 100 episodes reward is  : -88.88
Ep: 2400 last 100 episodes reward is  : -84.23
Ep: 2800 last 100 episodes reward is  : -84.96
Ep: 3200 last 100 episodes reward is  : -82.9
Ep: 3600 last 100 episodes reward is  : -83.52
Ep: 4000 last 100 episodes reward is  : -82.14
oh 0.0


score,▁
score,0.0


In [ ]:
'''
plt.plot(total_rewards)
plt.plot()
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.show() '''

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


"\nplt.plot(total_rewards)\nplt.plot()\nplt.xlabel('Episode')\nplt.ylabel('Reward')\nplt.show() "